# Evaluation of Reconstruction Performance

In [ ]:
import sys
import os
from os.path import join
parent_dir = os.path.abspath(join(os.getcwd(), os.pardir))
app_dir = join(parent_dir, "app")
if app_dir not in sys.path:
      sys.path.append(app_dir)

from pathlib import Path
import torch as pt
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from CNN_VAE import ConvDecoder, ConvEncoder, Autoencoder
from utils.training_loop import train_cnn_vae
import utils.config as config
from sklearn import metrics
import matplotlib.pyplot as plt

pt.manual_seed(0)

plt.rcParams["figure.dpi"] = 180

# use GPU if possible
device = pt.device("cuda:0") if pt.cuda.is_available() else pt.device("cpu")
print(device)

DATA_PATH = Path(os.path.abspath('')).parent / "data"
OUTPUT_PATH = Path(os.path.abspath('')).parent / "output" / "VAE" / "latent_study"

In [ ]:
# load test dataset
test_dataset = pt.load(join(DATA_PATH, "test_dataset.pt"))
print(len(test_dataset))

# split test dataset into the two flow conditions
X_test_1 = test_dataset[:len(test_dataset) / 2]        # ma0.84 alpha3.00 
X_test_2 = test_dataset[len(test_dataset) / 2:]        # ma0.84 alpha5.00

#### MSE and Variance Reconstruction with varying number of bottleneck neurons

In [ ]:
# Initialize lists to save the computed metrics
MSE_1 = []
MSE_2 = []
Var1 = []
Var2 = []

# compute the total variance test datasets
orig_Var1 = pt.var(X_test_1)
orig_Var2 = pt.var(X_test_2)

# iterate over all trained models with different latent size
for root, dirs, files in os.walk(OUTPUT_PATH):
    autoencoder = Autoencoder.load(OUTPUT_PATH)

#### Temporal MSE distribution with varying number of bottleneck neurons

#### Spatial MSE distribution with varying number of bottleneck neurons

#### Reconstructed pressure field compared to Ground Truth for two bottleneck sizes